In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from scipy import ndimage, misc
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('train_w_poster.csv')
test_df = pd.read_csv('test_w_poster.csv')

train_df = train_df[train_df['poster_path'].notnull()]
test_df = test_df[test_df['poster_path'].notnull()]
#change poster path accordingly
poster_prefix_path = 'Posters/'

train_df['poster_path'] = train_df['poster_path'].apply(lambda path: poster_prefix_path + path)
test_df['poster_path'] = test_df['poster_path'].apply(lambda path: poster_prefix_path + path)

In [5]:
def convert_jpg_array(poster_path):
    '''
    inputs: director of poster
    outputs: array of poster with shape 750,500,3
    '''
    img = Image.open(poster_path)
    img.save('/Users/Ihsaan Patel/Dropbox/5-Four Rooms.jpg')
    img = misc.imresize(img, (750, 500))
    arr = np.array(img)
    return arr

In [6]:
x = convert_jpg_array('/Users/Ihsaan Patel/Dropbox/HKS/Classes/CS 109B/Project/CS109B-Final-Project/Milestone_4/Posters/5-Four Rooms.jpg')
plt.imshow(x)

IOError: broken data stream when reading image file